In [3]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.offline as offline
import dash
from dash import dcc, html

In [4]:
etf_list = ['^RUT','^GSPC', '^STI','^FCHI', '^HSI','^NYA', '^AEX', '^TA125.TA', 'TA35.TA', '^N225', '^SSMI', '^IXIC', '^STOXX', 'GC=F', 'CL=F', 'SI=F', 'EURUSD=X', 'GBPUSD=X','JPY=X']


In [5]:
def get_all_data(list, date1, date2):
    all_etf_data = pd.DataFrame()
    for tkr_str in list:
        single_df = yf.download(tickers=tkr_str, start=date1, end=date2,interval='1d', auto_adjust=True)
        single_df['Ticker'] = tkr_str
        all_etf_data = pd.concat([all_etf_data, single_df])
    all_etf_data = all_etf_data.reset_index()
    all_etf_data.to_csv('all_etf')
    return all_etf_data

def calculate_sma(df, window_sizes=[20]):
    for w in window_sizes:
        df[f'SMA_{w}'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=w).mean())
    return df

def calculate_std(df, window_sizes=[20]):
    for w in window_sizes:
        df[f'STD_{w}'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=w).std())
    return df

def calculate_cci(data, n=14):
    # Calculate the Typical Price (TP)
    data["TP"] = (data["High"] + data["Low"] + data["Close"]) / 3

    # Calculate the Moving Average of TP
    data["TP_SMA"] = data.groupby('Ticker')["TP"].transform(lambda x: x.rolling(window=n).mean())

    # Calculate the Mean Deviation
    data["MD"] = data.groupby('Ticker')["TP"].transform(lambda x: x.rolling(window=n).apply(lambda y: abs(y - y.mean()).mean()))

    # Calculate the CCI
    data["CCI"] = (data["TP"] - data["TP_SMA"]) / (0.015 * data["MD"])

    return data


def calculate_rsi(df, n=14):
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(n).mean()
    avg_loss = loss.rolling(n).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['RSI'] = rsi
    return df



def calculate_rsi_for_all_etfs(df, n=14):
    rsi_df_list = []
    for etf in df['Ticker'].unique():
        etf_df = df[df['Ticker'] == etf]
        rsi = calculate_rsi(etf_df, n)
        rsi_df = pd.DataFrame({'Ticker': etf_df['Ticker'], 'Date': etf_df['Date'], 'RSI': rsi})
        rsi_df_list.append(rsi_df)
    rsi_df = pd.concat(rsi_df_list, ignore_index=True)
    return rsi_df
      

def makeit(df):
    df = calculate_sma(df)
    df = calculate_std(df)
    df = calculate_cci(df)

    modified_dfs = []
    for etf in df['Ticker'].unique():
        etf_df = df[df['Ticker'] == etf].copy()
        etf_df = calculate_rsi(etf_df)  # This will add RSI column to etf_df
        modified_dfs.append(etf_df)

    df = pd.concat(modified_dfs, ignore_index=True)
    df = df.dropna().reset_index(drop=True)
    df["BU"] = df['SMA_20'] + df['STD_20']*2
    df["BL"] = df['SMA_20'] - df['STD_20']*2  
    return df.copy()


In [69]:
all_etf_data = get_all_data(etf_list, '2013-01-01', '2023-01-01')
all_etf_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Date,Open,High,Low,Close,Volume,Ticker
0,2013-01-02,849.349976,873.989990,849.349976,873.419983,4202600000,^RUT
1,2013-01-03,873.830017,878.429993,870.760010,872.599976,3829730000,^RUT
2,2013-01-04,873.190002,880.469971,873.190002,879.150024,3424290000,^RUT
3,2013-01-07,876.340027,876.710022,873.739990,875.799988,3304970000,^RUT
4,2013-01-08,875.909973,876.320007,871.010010,874.700012,3601600000,^RUT
...,...,...,...,...,...,...,...
47563,2022-12-26,132.770004,133.237000,132.330002,132.789993,0,JPY=X
47564,2022-12-27,132.764008,133.587006,132.647995,132.764008,0,JPY=X
47565,2022-12-28,133.485992,134.395004,133.410995,133.485992,0,JPY=X
47566,2022-12-29,134.033997,134.188004,132.936005,134.033997,0,JPY=X


In [ ]:
# run this of you downloaded the data.
data = makeit(all_etf_data)
data

In [7]:
# skip download and open data
all_etf_data_csv = pd.read_csv('all_etf', usecols=[1,2,3,4,5,6,7])
data = makeit(all_etf_data_csv)


In [5]:
# Save df
data.to_csv("full_data")
data

,Date,Open,High,Low,Close,Volume,Ticker,SMA_20,STD_20,TP,TP_SMA,MD,CCI,RSI,BU,BL
0,2013-01-30,906.289978,906.700012,894.340027,896.909973,3726810000,^RUT,887.982996,11.718092,899.316671,892.308334,8.838334,52.863185,67.351396,911.419179,864.546812
1,2013-01-31,897.020020,902.900024,895.750000,902.090027,3999880000,^RUT,889.416498,11.595798,900.246684,893.730477,8.347077,52.043820,69.453281,912.608095,866.224901
2,2013-02-01,905.590027,912.760010,905.390015,911.200012,3836320000,^RUT,891.346500,11.858855,909.783346,895.838097,7.930547,117.228136,74.449600,915.064209,867.628790
3,2013-02-04,911.179993,911.179993,898.359985,899.280029,3390000000,^RUT,892.353000,11.621084,902.940002,897.509051,6.702995,54.015179,63.051210,915.595168,869.110832
4,2013-02-05,902.409973,909.609985,902.380005,908.219971,3618360000,^RUT,893.973999,11.450461,906.736654,899.303098,5.714592,86.720176,65.156583,916.874921,871.073077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47202,2022-12-26,132.770004,133.237000,132.330002,132.789993,0,JPY=X,135.848698,2.085348,132.785665,135.272429,1.661926,-99.754265,36.021425,140.019393,131.678003
47203,2022-12-27,132.764008,133.587006,132.647995,132.764008,0,JPY=X,135.550949,2.079074,132.999669,134.983310,1.763269,-74.998583,34.852262,139.709097,131.392801
47204,2022-12-28,133.485992,134.395004,133.410995,133.485992,0,JPY=X,135.287999,1.984250,133.763997,134.778333,1.761763,-38.383377,39.528271,139.256499,131.319499
47205,2022-12-29,134.033997,134.188004,132.936005,134.033997,0,JPY=X,135.121749,1.940407,133.719335,134.589429,1.724144,-33.643521,40.875619,139.002564,131.240935


In [8]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker', 'SMA_20',
       'STD_20', 'TP', 'TP_SMA', 'MD', 'CCI', 'RSI', 'BU', 'BL'],
      dtype='object')